## Install required dependency and SDK

In [78]:
!pip install --quiet google-cloud-bigquery google-cloud-aiplatform pandas

## Create the environment for the notebook
1. Create a source connection and grant IAM permissions (Steps from https://www.cloudskillsboost.google/course_templates/1210/labs/529948)

2. Generate embeddings


```
CREATE OR REPLACE MODEL `aurora_bay_faqs_conn.Embeddings`
REMOTE WITH CONNECTION `us.embedding_conn`
OPTIONS (ENDPOINT = 'text-embedding-005');
```

3. Created a Bigquery client so that I can directly use the Bigquery SQL syntax in the code. (ref: https://cloud.google.com/python/docs/reference/bigquery/latest/google.cloud.bigquery.client.Client#google_cloud_bigquery_client_Client_query)

In [79]:
from google.cloud import bigquery
from vertexai.preview.language_models import ChatModel
import vertexai
import pandas as pd

PROJECT_ID = "qwiklabs-gcp-04-d9bb68112d04"
LOCATION = "global"
BQ_DATASET = "aurora_bay_faqs_conn"
TABLE_RAW = "faq_data"
TABLE_EMBEDDED = "faq_data_embedded"
EMBED_MODEL = "faq_embeddings"
TABLE_ID = f"{PROJECT_ID}.{BQ_DATASET}.{TABLE_EMBEDDED}"
RAW_TABLE_ID = f"{PROJECT_ID}.{BQ_DATASET}.{TABLE_RAW}"
EMBED_MODEL_ID = f"{PROJECT_ID}.{BQ_DATASET}.{EMBED_MODEL}"

vertexai.init(project=PROJECT_ID, location=LOCATION)
bq_client = bigquery.Client(project=PROJECT_ID)
print("Environment set up.")

Environment set up.


## CSV upload to the BiqQuery table

1. Upload data from gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv table (faq_data)

In [80]:
uri = "gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv"
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
    write_disposition="WRITE_TRUNCATE"
)
load_job = bq_client.load_table_from_uri(uri, RAW_TABLE_ID, job_config=job_config)
load_job.result()
print(f"FAQ CSV loaded into BigQuery table: {RAW_TABLE_ID}")

FAQ CSV loaded into BigQuery table: qwiklabs-gcp-04-d9bb68112d04.aurora_bay_faqs_conn.faq_data



## Create Embedding Models
 1. Creates a Remote Model in BigQuery ML
This SQL command registers a remote Vertex AI model (text-embedding-005) as a BigQuery ML model so you can use it in ML.GENERATE_EMBEDDING() queries.

2. Uses a Preconfigured Connection
The REMOTE WITH CONNECTION clause refers to a BigQuery connection (us.embedding_conn) that links BigQuery to Vertex AI's embedding endpoint.

3. Executes SQL via Python Client
The Python code executes the SQL command using BigQuery's Python client and waits for it to complete using .result().

In [81]:
create_model_sql = f"""
CREATE OR REPLACE MODEL `{EMBED_MODEL_ID}`
REMOTE WITH CONNECTION `us.embedding_conn`
OPTIONS (ENDPOINT = 'text-embedding-005');
"""
bq_client.query(create_model_sql).result()
print("Remote embedding model created.")

Remote embedding model created.


## Generate Embeddings Using BigQuery ML

1. Generate Embeddings Using BigQuery ML produces an embedding vector using the model referenced.
2. Store Results in a New Table.
3. Execute via BigQuery Python Client.

# Note:
1. Since the loading of the csv we dont know what are the columns **string_field_0** and **string_field_1** is generated automatically.
2. Here
  1. **string_field_0** -  Question
  2. **string_field_1** - Answer

In [82]:
generate_embed_sql = f"""
CREATE OR REPLACE TABLE `{TABLE_ID}` AS
SELECT *, ml_generate_embedding_result AS embedding
FROM ML.GENERATE_EMBEDDING(
  MODEL `{EMBED_MODEL_ID}`,
  (
    SELECT CONCAT(string_field_0, ' ', string_field_1) AS content,
           string_field_0 AS question,
           string_field_1 AS answer
    FROM `{RAW_TABLE_ID}`
  )
);
"""
bq_client.query(generate_embed_sql).result()
print("Embeddings generated and stored.")

Embeddings generated and stored.


## Load Vector Search Results from BigQuery Table

1. This function queries the BigQuery table that stores VECTOR_SEARCH results.
2. It filters results by the current user question and returns top 3 most relevant rows

In [83]:
def fetch_faq_results(user_question):
    query = f"""
    SELECT
      query.query,
      result.base.question,
      result.base.answer,
      result.distance
    FROM VECTOR_SEARCH(
      TABLE `{TABLE_ID}`,
      'embedding',
      (
        SELECT
          ml_generate_embedding_result AS embedding,
          '{user_question}' AS query
        FROM ML.GENERATE_EMBEDDING(
          MODEL `{EMBED_MODEL_ID}`,
          (SELECT '{user_question}' AS content)
        )
      ),
      top_k => 3,
      options => '{{"fraction_lists_to_search": 1.0}}'
    ) AS result
    """
    return bq_client.query(query).to_dataframe()

## Gemini Chat Completion

1. This function uses the Gemini generative model to generate a chatbot response
2. It builds a prompt from top FAQ results retrieved from BigQuery

In [84]:
from vertexai.preview.generative_models import GenerativeModel
chat_model = GenerativeModel("gemini-2.0-flash-001")

def generate_bot_response(user_input):
    results = fetch_faq_results(user_input)
    context = "\n\n".join([f"Q: {row['question']}\nA: {row['answer']}" for _, row in results.iterrows()])
    prompt = f"You are a helpful assistant for the town of Aurora Bay. Use the following FAQ context to answer:\n\n{context}\n\nUser: {user_input}"
    response = chat_model.generate_content(prompt)
    return response.text

## Test the Chatbot
Send a sample question to the chatbot and display the response

In [85]:
while True:
    question = input("User: ")
    if question.strip().lower() in ["exit", "no", "quit"]:
        print("\nSession ended.")
        break
    print("\nAssistant:", generate_bot_response(question))

User: What are the policies in aurora bay?

Assistant: Could you please clarify what kind of policies you're interested in? Are you looking for fishing regulations, temperature information, recreational activities policies, or something else entirely? The more specific you are, the better I can assist you.

User: what are the fishing regulations in aurora bay?

Assistant: You can find the fishing regulations published by the Aurora Bay Fish & Game Department. They are located at 45 Coastal Drive. You can also find a digital copy of the guidelines on the town's official website.

User: what is the temperature range in aurora bay?

Assistant: Winters average between 10°F to 25°F, while summers are milder, around 50°F to 65°F. Keep in mind that temperatures can vary with coastal weather patterns.

User: exit

Session ended.
